# Story Generator - Winnie the Pooh 

## Part 2: Simple DSPy Retriever and Module

[1. Imports and environment](#1-imports-and-environment)

[2. Chroma retriever](#2-chroma-retriever)

[3. DSPy module](#3-dspy-module)

[4. Testing StoryGenerator](#4-testing-storygenerator)

### 1. Imports and environment

In [2]:
#pip install dspy-ai openai chromadb sentence_transformers

  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached optuna-4.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached structlog-24.4.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.3 kB)
  Using cached magicattr-0.1.6-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached torch-2.4.1-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pillow-10.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached pyarrow-17.0.0-cp311-cp311-macosx_11_0_arm64

In [7]:
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb
from chromadb.utils import embedding_functions
import dotenv
import os

# Establish paths
CHROMA_PATH = '../data/chroma_db'
DB_COLLECTION = "winnie_the_pooh"
default_ef = embedding_functions.DefaultEmbeddingFunction()

# Set up OpenAI API key
dotenv.load_dotenv()
#openai_key = os.getenv('OPENAI_API_KEY')

True

In [8]:
# List all collections in the Chroma database
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collections = chroma_client.list_collections()
print(collections)

[Collection(id=656e43d0-2738-4813-90d5-8b19c9a946b4, name=winnie_the_pooh)]


### 2. Chroma retriever

In [9]:
# Configure OpenAI as the language model
llm = dspy.OpenAI(model="gpt-3.5-turbo")

# Set up Chroma client and retriever
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = chroma_client.get_collection(DB_COLLECTION)

# Set up ChromadbRM as the retriever model
chroma_retriever = ChromadbRM(
    collection_name=DB_COLLECTION, 
    persist_directory=CHROMA_PATH, 
    embedding_function=default_ef, 
    k=10
    )

# Configure DSPy settings
dspy.settings.configure(lm=llm, rm=chroma_retriever)

In [10]:
#example of calling retriever
results = chroma_retriever("honey")
len(results)
results[0]

{'id': '9cb60098-9d48-4eb7-88d3-ad27219be56f',
 'score': 1.08407461643219,
 'long_text': 'then he got up, and said: "And the only reason for making honey is so as I can eat it." So he began to climb the tree. He climbed and he climbed and he climbed, and as he climbed he sang a little',
 'metadatas': {'author': 'A. A. Milne',
  'chapter': 1.0,
  'chunk': 17.0,
  'title': 'Winnie the Pooh'}}

### 3. DSPy module

In [19]:

class GenerateStory(dspy.Signature):
    """Generate a Winnie the Pooh style story."""
    context = dspy.InputField(desc="relevant passages from Winnie the Pooh stories")
    prompt = dspy.InputField()
    story = dspy.OutputField()

class StoryGenerator(dspy.Module):
    def __init__(self, chroma_retriever):
        super().__init__()
        self.retriever = chroma_retriever
        self.generate = dspy.ChainOfThought(GenerateStory)

    def forward(self, prompt):
        retrieved = self.retriever(prompt, k=5)
        context = "\n".join([doc.long_text for doc in retrieved])

        result = self.generate(context=context, prompt=prompt)
        return dspy.Prediction(story=result.story)


### 4. Testing StoryGenerator 

In [21]:
# Create an instance of the StoryGenerator
story_gen = StoryGenerator(chroma_retriever)

new_story = story_gen("Winnie the Pooh and friends go on a picnic")
print(new_story)

#

Prediction(
    story='One sunny day in the Hundred Acre Wood, Winnie the Pooh woke up with a rumble in his tummy. He thought to himself, "What a perfect day for a picnic!" He quickly made his way to Piglet\'s house to invite his friends to join him. Piglet, Tigger, Rabbit, Eeyore, and even Owl were all excited at the idea of a picnic.\n\nThey all gathered their favorite snacks and treats, from honey pots to carrots and nuts, and set off'
)


Notes for tweaks and improvements: 

- define dspy.Signature to more clearly specify how to story should be set up
- the stories do not seem to be complete. Hoping Signature will fix this
- Change chunking in processing to complete sentences or stop at punctuations
- make sure stories end in complete sentences


Other functionality or set up: 

- User can provie name of the main character or settings outside of existing stories. 